In this lab you will implement the **skip-gram model with negative sampling (SGNS)** from Lecture&nbsp;1.4, and use it to train word embeddings on the text of the [Simple English Wikipedia](https://simple.wikipedia.org/wiki/Main_Page).

⚠️ The dataset for this lab contains 18M tokens. This is very little as far as word embedding datasets are concerned – for example, the original word2vec model was pre-trained on 100B tokens. In spite of this, you will need to think about efficiency when processing the data and training your models. In particular, wherever possible you should use iterators rather than lists, and vectorize operations (using [NumPy](https://numpy.org) or [PyTorch](https://pytorch.org)) as much as possible.

# L1: Word representations

## Load the data

The data for this lab comes as a bz2-compressed plain text file. It consists of 1,163,769 sentences, with one sentence per line and tokens separated by spaces. The cell below contains a wrapper class `SimpleWikiDataset` that can be used to iterate over the sentences (lines) in the text file. On the Python side of things, each sentence is represented as a list of tokens (strings).

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import bz2

class SimpleWikiDataset():
    
    def __init__(self, max_sentences=None):
        self.max_sentences = max_sentences
    
    def __iter__(self):
        with bz2.open('/content/drive/MyDrive/TDDE09/labs/l1/simplewiki.txt.bz2', 'rt', encoding='utf-8') as sentences: # unzip manually if it is not working
            for i, sentence in enumerate(sentences):
                if self.max_sentences and i >= self.max_sentences:
                    break
                yield sentence.split()

Using this class, we define two variants of the dataset: the full dataset and a minimal version with the first 1% of the sentences in the full dataset. The latter will be useful to test code without running it on the full dataset.

In [7]:
# Dataset with all sentences (N = 1,163,769)
full_dataset = SimpleWikiDataset()

# Minimal dataset
mini_dataset = SimpleWikiDataset(max_sentences=11638)

CURR_DATASET = full_dataset

The next code cell defines a generator function that allows you to iterate over all tokens in a dataset:

In [8]:
def tokens(sentences):
    for sentence in sentences:
        for token in sentence:
            yield token

To illustrate how to use this function, here is code that prints the number of tokens in the full dataset:

In [9]:
print(sum(1 for t in tokens(full_dataset)))

17594885


## Problem 1: Build the vocabulary and frequency table

Your first task is to construct the embedding **vocabulary** – the set of unique words that will receive an embedding. Because you will eventually need to map words to vector dimensions, you will represent the vocabulary as a dictionary that maps words (strings) to a contiguous range of integers.

Along with the vocabulary, you will also construct the **frequency table**, that is, the table that holds the absolute frequencies (counts) in the data, for all words in your vocabulary. This will simply be an array of integers, indexed by the word ids in the vocabulary.

To construct the vocabulary and the frequency table, complete the skeleton code in the cell below:

In [10]:
import numpy as np

def make_vocab_and_counts(sentences, min_count=5):
    vocab = {}        # The vocabulary to be returned.
    counts = {}       # For simple count of all words without index.
    freq_table = []   # The frequenct table to be returned. Converted to np array later.

    for sentence in sentences:
      for token in sentence:
        if token not in counts:
          counts[token] = 1
        else:
          counts[token] += 1

    for sentence in sentences: 
      for token in sentence:
        idx = len(vocab)
        if token not in vocab and counts[token] >= min_count:
          vocab[token] = idx
          freq_table.append(counts[token])

    return vocab, np.array(freq_table)

Your code should comply with the following specification:

**make_vocab_and_counts** (*sentences*, *min_count* = 5)

> Reads from an iterable of *sentences* (lists of string tokens) and returns a pair *vocab*, *counts* where *vocab* is a dictionary representing the vocabulary and *counts* is a 1D-[ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) with the absolute frequencies (counts) of the words in the vocabulary. The dictionary *vocab* maps words to a contiguous range of integers starting at&nbsp;0. In the *counts* array, the entry at index $i$ is the count of that word in *vocab* which maps to $i$. Words that occur less than *min_count* times are excluded from the vocabulary.

### 🤞 Test your code

To test your code, print the sizes of the vocabularies constructed from the two datasets, as well as the count totals. The correct vocabulary size for the minimal dataset is 3,231; for the full dataset, the correct vocabulary size is 73,339. The correct totals are 155,818 for the minimal dataset and 17,297,355 for the full dataset.

In [11]:
# TODO: Test your code here
vocab, freq_table = make_vocab_and_counts(CURR_DATASET)
print(vocab['usually'])
print(len(freq_table))
print(len(vocab))
#print(vocab)
print(sum(freq_table)) # Correct!! :)

3
73339
73339
17297355


## Problem 2: Preprocess the data

Your next task is to preprocess the training data. This involves the following:

* Discard words that are not in the vocabulary
* Map each word to its vocabulary id
* Randomly discard words according to the subsampling strategy covered in Lecture&nbsp;1.4
* Discard sentences that have become empty

As a reminder, the subsampling strategy involves discarding tokens $w$ with probability

$$
P(w) = \max (0, 1-\sqrt{tN/\#(w)})
$$

where $\#(w)$ is the count of $w$, $N$ is the total number of counts, and $t$ is the chosen threshold (default value: 0.001).

The cell below contains skeleton code for a generator function `preprocess`:

In [16]:
import torch
import math

def discard(N, dist, w, t = 0.001):
  pw = max(0, 1 - math.sqrt(t*N / w))
  
  if pw >= np.random.rand(1):
    return True

def preprocess(vocab, counts, sentences, threshold=0.001):
    # TODO: Replace the next line with your own code
    N = sum(counts)
    cumulative_sums = torch.cumsum(torch.from_numpy(counts), dim=0)

    for sentence in sentences:
      list_of_ids = []
      for word in sentence:
        if word in vocab and not discard(N, cumulative_sums, counts[vocab[word]], threshold):          
          list_of_ids.append(vocab[word])

      if list_of_ids:
        yield list_of_ids
# Worst case for discarding randomly is that the whole sentence is gone, need to be implemented?

Extend this skeleton code into a function that implements the preprocessing. Your code should comply with the following specification:

**preprocess** (*vocab*, *counts*, *sentences*, *threshold* = 0.001)

> Reads from an iterable of *sentences* (lists of string tokens) and yields the preprocessed sentences as non-empty lists of word ids (integers). Words not in *vocab* are discarded. The remaining words are randomly discarded according to the subsampling strategy with the given *threshold*. In the non-empty sentences, each token is replaced by its id in the vocabulary.

**⚠️ Please observe** that your function should *yield* the preprocessed sentences, not return a list with all of them. That is, we ask you to write a *generator function*. If you have not worked with generators and iterators before, now is a good time to read up on them. [More information about generators](https://wiki.python.org/moin/Generators)

### 🤞 Test your code

Test your code by comparing the total number of tokens in the preprocessed version of each dataset with the corresponding number for the original data. The former should be ca. 59% of the latter for the minimal dataset, and ca. 69% for the full dataset. The exact percentage will vary slightly because of the randomness in the sampling. You may want to repeat your computation several times.

In [34]:
processed = [sentence_list for sentence_list in preprocess(vocab, freq_table, CURR_DATASET)]
print(sum(1 for t in tokens(processed)))
print(sum(1 for t in tokens(CURR_DATASET)))

print(sum(1 for t in tokens(processed)) / sum(1 for t in tokens(CURR_DATASET)))


12221369
17594885
0.694597833404424


## Problem 3: Generate the training examples

Your next task is to translate the preprocessed sentences into training examples for the skip-gram model: both *positive examples* (target word–context word pairs actually observed in the data) and *negative examples* (pairs randomly sampled from a noise distribution).

**⚠️ We expect that solving this problem will take you the longest time in this lab.**

### General strategy

The general plan for solving this problem is to implement a generator function that traverses the preprocessed sentences, at each position of the text samples a window, and then extracts all positive examples from it. For each positive example, the function also generates $k$ negative examples, where $k$ is a hyperparameter. Finally, all examples (positive and negative) are combined into the tensor representation described below.

### Representation

How should you represent a batch of training examples? Writing $B$ for the batch size, the obvious choice would be to represent the inputs as a matrix of shape $[B, 2]$ and the output labels (positive/negative) as a vector of length $B$. This representation would be quite wasteful on the input side, however, as each target word (index) from a positive example would have to be repeated in all negative samples. For example ($k=3$):

Here you will use a different representation: First, instead of a single input batch, there will be a *pair* of input batches – a vector for the target words and a matrix for the context words. If the target word vector has length $B$, the context word matrix has shape $[B, 1+k]$. The $i$th element of the target word vector is the target word for *all* context words in the $i$th row of the context word matrix: the first column of that row comes from a positive example, the remaining columns come from the $k$ negative samples. Accordingly, the batch with the output labels will be a matrix of the same shape as the context word matrix, with its first column set to&nbsp;1 and its remaining columns set to&nbsp;0. Corresponding to the example above:

For the present problem, you will only be concerned with the two input batches; the output batch will be constructed in the training procedure. In fact, for a fixed batch size $B$, that batch is always exactly the same, so you will only have to build it once.

### Negative sampling

Recall from Lecture&nbsp;1.4 that the probability of a word $c$ to be selected as the context word in a negative sample is proportional to its exponentiated count $\#(c)^\alpha$, where $\alpha$ is a hyperparameter (default value: 0.75).

To implement negative sampling from this distribution, you can follow a standard recipe: Start by pre-computing an array containing the *cumulative sums* of the exponentiated counts. Then, generate a random cumulative count $n$, and find that index in the pre-computed array at which $n$ should be inserted to keep the array sorted. That index identifies the sampled context word.

All operations in this recipe can be implemented efficiently in PyTorch; the relevant functions are [`torch.cumsum`](https://pytorch.org/docs/stable/generated/torch.cumsum.html) and [`torch.searchsorted`](https://pytorch.org/docs/stable/generated/torch.searchsorted.html). For optimal efficiency, you should sample all $B \times k$ negative examples in a batch at once.

Here is skeleton code for this problem:

In [28]:
from torch import LongTensor
import math as m

def get_c1_c2(sent, c_w, t_id):
  for context_id in sent[c_w]: # 3
    if t_id != context_id:
      yield t_id, [context_id]


def training_examples(vocab, counts, sentences, window=5, num_ns=5, batch_size=1<<19, ns_exponent=0.75):
    # TODO: Replace the next line with your own code
    cumulative_sum = torch.cumsum(torch.from_numpy(counts**ns_exponent), dim=0)
    rand = torch.rand(batch_size, num_ns)*cumulative_sum[-1]
    negative_samples = torch.searchsorted(cumulative_sum, rand)


    preprocessed = [sentence for sentence in preprocess(vocab, freq_table, sentences)]
    comp1 = []
    comp2 = []

    for sentence in preprocessed: # 1
      window_size =  int(np.random.uniform(2, window))
      
      for index, target_id in enumerate(sentence): # 2
        context_window = slice(max(0, -window_size+index), min(window_size+index+1, len(sentence)),1)

        for f, s in get_c1_c2(sentence, context_window, target_id):
          comp1.append(f)
          comp2.append(s)
          if len(comp1) == batch_size:
            yield torch.tensor(comp1), torch.cat((torch.tensor(comp2), negative_samples), 1)
            comp1 = []
            comp2 = []
            negative_samples = torch.searchsorted(cumulative_sum, torch.rand(batch_size, num_ns)*cumulative_sum[-1])

    yield torch.tensor(comp1), torch.cat((torch.tensor(comp2), negative_samples[:len(torch.tensor(comp1))]), 1)

Your code should comply with the following specification:

**training_examples** (*vocab*, *counts*, *sentences*, *window* = 5, *num_ns* = 5, *batch_size* = 524,288, *ns_exponent*=0.75)

> Reads from an iterable of *sentences* (lists of string tokens), preprocesses them using the function implemented in Problem&nbsp;2, and then yields pairs of input batches for gradient-based training, represented as described above. Each batch contains *batch_size* positive examples. The parameter *window* specifies the maximal distance between a target word and a context word in a positive example; the actual window size around any given target word is sampled uniformly at random. The parameter *num_ns* specifies the number of negative samples per positive sample. The parameter *ns_exponent* specifies the exponent in the negative sampling (called $\alpha$ above).

### 🤞 Test your code

To test your code, compare the total number of positive samples (across all batches) to the total number of tokens in the (un-preprocessed) minimal dataset. The ratio between these two values should be ca. 2.64. If you can spare the time, you can make the same comparison on the full dataset; here, the expected ratio is 3.25. As before, the numbers may vary slightly because of randomness, so you may want to run the comparison more than once.

In [29]:
# TODO: Test your code here
original_length = sum([len(token) for token in CURR_DATASET])
print("Original length: ", original_length)

n_batches = 0
examples = 0
for comp1, comp2 in training_examples(vocab, freq_table, list(CURR_DATASET)):
  print("Batch: ", n_batches)
  n_batches += 1
  examples += len(comp1)

print("n_examples: ", examples)

print("Ratio: ", str(examples / original_length))


Original length:  17594885
Batch:  0
Batch:  1
Batch:  2
Batch:  3
Batch:  4
Batch:  5
Batch:  6
Batch:  7
Batch:  8
Batch:  9
Batch:  10
Batch:  11
Batch:  12
Batch:  13
Batch:  14
Batch:  15
Batch:  16
Batch:  17
Batch:  18
Batch:  19
Batch:  20
Batch:  21
Batch:  22
Batch:  23
Batch:  24
Batch:  25
Batch:  26
Batch:  27
Batch:  28
Batch:  29
Batch:  30
Batch:  31
Batch:  32
Batch:  33
Batch:  34
Batch:  35
Batch:  36
Batch:  37
Batch:  38
Batch:  39
Batch:  40
Batch:  41
Batch:  42
Batch:  43
Batch:  44
Batch:  45
Batch:  46
Batch:  47
Batch:  48
Batch:  49
Batch:  50
Batch:  51
Batch:  52
Batch:  53
Batch:  54
Batch:  55
Batch:  56
Batch:  57
Batch:  58
Batch:  59
Batch:  60
Batch:  61
Batch:  62
Batch:  63
Batch:  64
Batch:  65
Batch:  66
Batch:  67
Batch:  68
Batch:  69
Batch:  70
Batch:  71
Batch:  72
Batch:  73
Batch:  74
Batch:  75
Batch:  76
Batch:  77
Batch:  78
Batch:  79
Batch:  80
Batch:  81
Batch:  82
Batch:  83
Batch:  84
Batch:  85
Batch:  86
Batch:  87
Batch:  88
Batc

In [ ]:
sum(1 for t in tokens(CURR_DATASET))

## Problem 4: Implement the model

Now it is time to implement the skip-gram model as such. The cell below contains skeleton code for this. As you will recall from Lecture&nbsp;1.4, the core of the implementation is formed by two embedding layers: one for the target word representations, and one for the context word representations. Your task is to implement the missing `forward()` method.

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SGNSModel(nn.Module):
    
    def __init__(self, vocab, embedding_dim):
        super().__init__()
        self.vocab = vocab
        self.w = nn.Embedding(len(vocab), embedding_dim)
        self.c = nn.Embedding(len(vocab), embedding_dim)
    
    def forward(self, w, c):
        # TODO: Replace the next line with your own code
        return torch.bmm(torch.unsqueeze(self.w(w), 1), torch.transpose(self.c(c), 1, 2)) # can be one line of code. FOUND IT!! :))))

Your implementation of the `forward()` method should comply with the following specification:

**forward** (*self*, *w*, *c*)

> The input to this methods is a tensor *w* with target words of shape $[B]$ and a tensor *c* with context words of shape $[B, 1+k]$, where $B$ is the batch size and $k$ is the number of negative samples. The two tensors are structured as explained for Problem&nbsp;3. The output of the method is a tensor $D$ of shape $[B, k+1]$ where entry $D_{ij}$ is the dot product between the embedding vector for the $i$th target word and the embedding vector for the context word in row $i$, column $j$.

**💡 Hint:** To compute a dot product $x^\top y$, you can first compute the Hadamard product $z = x \odot y$ and then sum up the elements of $z$.

### 🤞 Test your code

Test your code by creating an instance of the model, and check that `forward` returns the expected result on random input tensors *w* and *c*. To help you, the following function will return a random example from the first 100 examples produced by `training_examples`.

In [31]:
import numpy as np

def random_example(vocab, counts, sentences):
    skip = np.random.randint(100)
    for i, example in enumerate(training_examples(vocab, counts, sentences, num_ns=1, batch_size=5)):
        if i >= skip:
            break
    return example


In [22]:
w, c = random_example(vocab, freq_table, CURR_DATASET)
SGNS = SGNSModel(vocab, 50)
SGNS.forward(w,c)

tensor([[[-6.5494, -5.7961]],

        [[-7.5001,  7.7565]],

        [[ 5.2542,  4.6341]],

        [[ 0.7494,  7.5003]],

        [[-1.5652,  7.2306]]], grad_fn=<BmmBackward0>)

## Problem 5: Train the model

Once you have a working model, it is time to train it. The training loop for the skip-gram model will be very similar to the prototypical training loop that you saw in Lecture&nbsp;0.6, with two things to note:

First, instead of categorical cross entropy, you will use binary cross entropy. Just like the standard implementation of the softmax classifier, the skip-gram model does not include a final non-linearity, so you should use [`binary_cross_entropy_with_logits()`](https://pytorch.org/docs/1.9.1/generated/torch.nn.functional.binary_cross_entropy_with_logits.html).

The second thing to note is that you will have to create the tensor with the output labels, as explained already in Problem&nbsp;3. This should be a matrix of size $[B, 1+k]$ whose first column contains $1$s and whose remaining columns contains $0$s.

Here is skeleton code for the training loop, including default values for the most important hyperparameters:

In [32]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import tqdm

def output_labels(batch, num_ns):
    tensor = torch.LongTensor(batch, num_ns + 1)
    tensor[:,0] = 1
    tensor[:, 1:] = 0
    return tensor

def train(sentences, embedding_dim=50, window=5, num_ns=5, batch_size=1<<19, n_epochs=1, lr=1e-1):
    # Create the vocabulary and the counts
    vocab, counts = make_vocab_and_counts(sentences)
    
    # Initialize the model
    model = SGNSModel(vocab, embedding_dim)
    
    # Initialize the optimizer. Here we use Adam rather than plain SGD
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # TODO: Add your code here
    train_losses = []


    # Inspired by Lab1_A example notebook
    with tqdm.tqdm(total=n_epochs) as pbar:
        for t in range(n_epochs):
            pbar.set_description(f'Epoch {t+1}')
            iteration = 0
            running_loss = 0

            # Start training
            model.train()
            running_loss = 0
            for bx, by in training_examples(vocab, counts, sentences):
                optimizer.zero_grad()
                output = model.forward(bx, by)
                by = torch.unsqueeze(output_labels(by.shape[0], num_ns), 1)
                loss = F.binary_cross_entropy_with_logits(output.to(torch.float32), by.to(torch.float32))
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                iteration += 1
                
            pbar.update()
            train_losses.append(running_loss / len(vocab))
            print("\n[Epoch " + str(t+1) +  " avg loss]: " + str(running_loss/iteration))

    return model

To show you how `train` is meant to be used, the code in the next cell trains a model on the minimal dataset.

### 🤞 Test your code

Test your implementation of the training loop by training a model on the minimal dataset. This should only take a few seconds. You will not get useful word vectors, but you will be able to see whether your code runs without errors.

Once you have passed this test, you can train a model on the full dataset. Print the loss to check that the model is actually learning; if the loss is not decreasing, try to find the problem before wasting time (and energy) on useless training.

Training on the full dataset will take some time – on a CPU, you should expect 10–40 minutes per epoch, depending on hardware. To give you some guidance: The total number of positive examples is approximately 58M, and the batch size is chosen so that each batch contains roughly 10% of these examples. To speed things up, you can train using a GPU; our reference implementation runs in less than 2 minutes per epoch on [Colab](http://colab.research.google.com).

In [33]:
# TODO: Train your model on the full dataset here

model_big = train(CURR_DATASET, n_epochs = 10)

Epoch 2:  10%|█         | 1/10 [05:24<48:42, 324.76s/it]


[Epoch 1 avg loss]: 0.5989209025033883


Epoch 3:  20%|██        | 2/10 [10:45<43:00, 322.56s/it]


[Epoch 2 avg loss]: 0.40541314147412777


Epoch 4:  30%|███       | 3/10 [16:04<37:25, 320.82s/it]

...


[Epoch 8 avg loss]: 0.3914105822997434


Epoch 10:  90%|█████████ | 9/10 [47:56<05:18, 318.34s/it]


[Epoch 9 avg loss]: 0.3909832313656807


Epoch 10: 100%|██████████| 10/10 [53:12<00:00, 319.25s/it]


[Epoch 10 avg loss]: 0.3913383702082293


## Problem 6: Analyse the embeddings (reflection)

Now that you have a trained model, you will probably be curious to see what it has learned. You can inspect your embeddings using the [Embedding Projector](http://projector.tensorflow.org). To that end, click on the ‘Load’ button, which will open up a dialogue with instructions for how to upload embeddings from your computer.

You will need to upload two tab-separated files. To create them, you can use the following code:

In [26]:
def save_model(model):
    # Extract the embedding vectors as a NumPy array
    embeddings = model.w.weight.detach().numpy()
    
    # Create the word–vector pairs
    items = sorted((i, w) for w, i in model.vocab.items())
    items = [(w, e) for (i, w), e in zip(items, embeddings)]
    
    # Write the embeddings and the word labels to files
    with open('vectors.tsv', 'wt') as fp1, open('metadata.tsv', 'wt') as fp2:
        for w, e in items:
            print('\t'.join('{:.5f}'.format(x) for x in e), file=fp1)
            print(w, file=fp2)

Take some time to explore the embedding space. In particular, inspect the local neighbourhoods of words that you are curious about, say the 10 closest neighbours. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* Which words did you try? Which results did you get? Did you do anything else than inspecting local neighbourhoods?
* Based on what you know about word embeddings, did you expect your results? How do you explain them?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

*TODO: Enter your text here*


Words tried:

Woman:landon 0.414
airoldi 0.445
sudeep 0.454
westbound 0.474
cambodian 0.481
streeterville 0.482
griggs 0.483
tur 0.486
frankweiler 0.490
egocentric 0.496

Sister:
rubingh 0.455
fremantle 0.467
pulling 0.485
hindered 0.485
muller 0.494
defencemen 0.500
peanut 0.500
numbness 0.504
lyrebird 0.507
nie 0.513

Dog: 
explosions 0.387
telephones 0.467
motorized 0.470
carthaginian 0.476
shikasta 0.485
beveren 0.509
spirituality 0.510
destroys 0.522
napoléon 0.523
amplification 0.523

Question1
The results show poor coorelation. Sister should be neighbours with something semantically similar. The words given could never be guessed as an relation to the word checked in the word-space. I.e., sister has the neighbours peanut and pulling, dog has the neighbours explosions and telephones. 
We also tried with the small dataset and this gave the same results but with different neighbours.

Besides inspecting the local neighbours we inspected the 100 closests neighbours. For the word "football" one can see that the training does not yield the results expected, out of all the words 1 single word "coach" could be related to the word football. The rest of the words seems more or less random

Question2
From our knowledge the embeddings should correlate words that are close to eachother and dissimilate words that are not the same as the word. Fully training the model should yield neighbours that are semantically related. There is something wrong when correlating the words (the context words). This can be seen because the words in the space that are neighbouring are not at all similar.

Question3
We have learned the skipgram model for learning word embeddings. This learning is done by applying a context window to the sentences in the corpus and adding negative samples that have no relation to the word. The main learning point for this lab for us was handling torch and manipulating tensors/vectors/lists etc. 
The model-parts such as subsampling and negative sampling was mainly learned by deeply examining the lectures and models, whilst the torch learning part was done by examining torch sizes and understanding the error prompts given when wrongly coded. This learning matters to get a deep knowledge of the models used. For instance, the word2vec models can easily be used from libaries such as sklearn, but what negative sampling and randomly discarding words based on frequency is does not really show when using these pre-done models. 




👍 Well done!